In [ ]:
# Iván Andrés Trujillo Abella
# ivantrujillo1229@gmail.com

In [ ]:
# update scoring metric also in ridge, and linear...

In [ ]:
# The dataset is used from sklearn v1.1

In [ ]:
# Recursive feature selection
# The first iteration consider all variables.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.feature_selection import RFECV
from sklearn.metrics import fbeta_score, make_scorer
from sklearn.metrics import mean_absolute_percentage_error as MAPE
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error as MSE


url = "https://raw.githubusercontent.com/it-ces/Datasets/refs/heads/main/load_boston-scikit_v1.csv"
df= pd.read_csv(url)


In [ ]:
X = df.drop(columns='Price')
y = df['Price']

In [ ]:
linear_model = LinearRegression().fit(X,y)
rfe = RFECV(
estimator = linear_model,
step = 1,
cv = 5,
scoring =make_scorer(MAPE, greater_is_better=False),
                     min_features_to_select = 1
)
rfe.fit(X,y)

RFECV(cv=5, estimator=LinearRegression(),
      scoring=make_scorer(mean_absolute_percentage_error, greater_is_better=False, response_method='predict'))

In [ ]:
X.columns[rfe.support_] # leakage information!

Index(['CHAS', 'NOX', 'RM', 'DIS', 'PTRATIO', 'LSTAT'], dtype='object')

In [ ]:
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm

In [ ]:
transform = StandardScaler()
X_ = transform.fit_transform(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_, y ,
                                                    test_size = 0.25,
                                                    shuffle=True,
                                                    random_state=132)

In [ ]:
OLS = sm.OLS(y_train, X_train) # Definimos el modelo del robot
OLS_model = OLS.fit() # sin regularización
ridge = OLS.fit_regularized(method='elastic_net', L1_wt=0,
alpha=0.25, refit=True) # ridge
lasso = OLS.fit_regularized(method='elastic_net', L1_wt=1,
                            alpha=0.01, refit =True) # lasso

In [ ]:
print(MSE(y_test, OLS_model.predict(X_test)) / y_test.shape[0])
print(MSE(y_test, ridge.predict(X_test)) / y_test.shape[0] )
print(MSE(y_test, lasso.predict(X_test)) / y_test.shape[0])

4.239030495660287
4.083348194153284
4.239030495660287


In [ ]:
coef = pd.DataFrame(OLS_model.params, columns=['OLS'])
coef['lasso'] = lasso.params
coef['ridge'] = ridge.params
coef

,OLS,lasso,ridge
x1,-1.625194,-1.625194,-1.108566
x2,2.574047,2.574047,1.419429
x3,-0.133123,-0.133123,-0.370939
x4,1.147023,1.147023,1.098452
x5,-3.000656,-3.000656,-1.101607
x6,0.873346,0.873346,1.798575
x7,1.835129,1.835129,0.713693
x8,-4.415183,-4.415183,-2.138508
x9,1.668155,1.668155,0.328457
x10,-0.696185,-0.696185,-0.209736


In [ ]:
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import KFold
from sklearn import linear_model

In [ ]:
np.linspace(0,100, num=10000)

array([0.00000000e+00, 1.00010001e-02, 2.00020002e-02, ...,
       9.99799980e+01, 9.99899990e+01, 1.00000000e+02])

In [ ]:
from sklearn.linear_model import Lasso

In [ ]:
# una clase....
def grid_lasso(X_train, y_train):
    model =  linear_model.Lasso(random_state=123)
    alpha = np.linspace(0,1, 100)
    grid = dict(alpha = alpha)
    cv = KFold(n_splits=5, shuffle=True, random_state=10) # replicables...
    grid_search = GridSearchCV(estimator=model,
                               param_grid=grid,
                               cv=cv,
                               scoring= 'neg_mean_squared_error',
                               )
    grid_result = grid_search.fit(X_train, y_train)
    return grid_search.best_estimator_

In [ ]:
best_model = grid_lasso(X_train, y_train) #entreno el modelo con X_train y y_train
preds_ = best_model.fit(X_train,y_train).predict(X_test)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:1473: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  return fit_method(estimator, *args, **kwargs)
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.643e+03, tolerance: 2.767e+00 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent

In [ ]:
print(mean_squared_error(y_test, preds_), best_model.alpha)
model = linear_model.Lasso(random_state=123, alpha=0.001)
preds2 = model.fit(X_train,y_train).predict(X_test)
print(mean_squared_error(y_test, preds2), 'este siepre tiene que ser mayor siempre y cuando sea diferente del alpha optimo')

18.375387178973504 0.010101010101010102
18.481638871640627 este siepre tiene que ser mayor siempre y cuando sea diferente del alpha optimo


In [ ]:
# Aquí es muy importante la estandarización!
# Standardization take in mind lekeage information!!
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
model.fit(X_train, y_train)
y_pred= model.predict(X_test)
print(mean_squared_error(y_test, y_pred))